In [2]:
from transformers import logging
logging.set_verbosity_error()

In [3]:
from datasets import load_dataset
from peft import LoraConfig
from trl import SFTTrainer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
    logging,
)

2024-10-03 07:13:25.309573: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-03 07:13:25.327380: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-03 07:13:25.349106: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-03 07:13:25.355855: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-03 07:13:25.371556: I tensorflow/core/platform/cpu_feature_guar

In [4]:
base_model = "amd/AMD-Llama-135m" # https://huggingface.co/meta-llama/Llama3.1-8B

hf_dataset = "ahmeterdempmk/Llama-E-Commerce-Fine-Tune-Data" # https://huggingface.co/ahmeterdempmk/Llama-E-Commerce-Fine-Tune-Data

In [5]:
dataset = load_dataset(hf_dataset, split="train")

In [6]:
model = AutoModelForCausalLM.from_pretrained (
    base_model,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1
model.low_cpu_mem_usage=True

In [7]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [8]:
peft_params = LoraConfig (
    lora_alpha=16, # the scaling factor for the low-rank matrices
    lora_dropout=0.1, # the dropout probability of the LoRA layers
    r=64, # the dimension of the low-rank matrices
    bias="none",
    task_type="CAUSAL_LM", # the task to train for (sequence-to-sequence language modeling in this case)
)

In [9]:
training_params = TrainingArguments (
    output_dir="./Llama135Results",
    num_train_epochs=5, # One training epoch.
    per_device_train_batch_size=4, # Batch size per GPU for training.
    gradient_accumulation_steps=1, # This refers to the number of steps required to accumulate the gradients during the update process.
    optim="paged_adamw_32bit", # Model optimizer (AdamW optimizer).
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4, # Initial learning rate. (Llama 3.1 8B ile hesaplandı)
    weight_decay=0.001, # Weight decay is applied to all layers except bias/LayerNorm weights.
    fp16=False, # Disable fp16/bf16 training.
    bf16=False, # Disable fp16/bf16 training.
    max_grad_norm=0.3, # Gradient clipping.
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [10]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="input",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [11]:
train_output = trainer.train()

{'loss': 5.3823, 'grad_norm': 1.848921537399292, 'learning_rate': 0.0002, 'epoch': 0.3125}
{'loss': 3.4219, 'grad_norm': 3.317167282104492, 'learning_rate': 0.0002, 'epoch': 0.625}
{'loss': 2.9662, 'grad_norm': 2.463834285736084, 'learning_rate': 0.0002, 'epoch': 0.9375}
{'loss': 2.6371, 'grad_norm': 5.358040809631348, 'learning_rate': 0.0002, 'epoch': 1.25}
{'loss': 2.3203, 'grad_norm': 1.7727932929992676, 'learning_rate': 0.0002, 'epoch': 1.5625}
{'loss': 2.2108, 'grad_norm': 1.6899242401123047, 'learning_rate': 0.0002, 'epoch': 1.875}
{'loss': 2.1639, 'grad_norm': 1.7333773374557495, 'learning_rate': 0.0002, 'epoch': 2.1875}
{'loss': 2.1299, 'grad_norm': 3.5865256786346436, 'learning_rate': 0.0002, 'epoch': 2.5}
{'loss': 2.0782, 'grad_norm': 1.7934837341308594, 'learning_rate': 0.0002, 'epoch': 2.8125}
{'loss': 1.8642, 'grad_norm': 1.9956821203231812, 'learning_rate': 0.0002, 'epoch': 3.125}
{'loss': 1.7105, 'grad_norm': 2.5094962120056152, 'learning_rate': 0.0002, 'epoch': 3.4375}


In [11]:
#text = "soy beans for your healty and delicious meals"
#prompt = f"""
#You are extracting product title and description from given text and rewriting the description and enhancing it when necessary.
#Always give response in the user's input language.
#Always answer in the given json format. Do not use any other keywords. Do not make up anything.
#Explanations should contain at least three sentences each.

#Json Format:
#{{
#"title": "<title of the product>",
#"description": "<description of the product>"
#}}

#Examples:

#Product Information: Rosehip Marmalade, keep it cold
#Answer: {{"title": "Rosehip Marmalade", "description": "You should store this delicisious roseship marmelade in cold conditions. You can use it in your breakfasts and meals."}}

#Product Information: Blackberry jam spoils in the heat
#Answer: {{"title": "Blackberry Jam", "description": "Please store it in cold conditions. Recommended to be consumed at breakfast. Very sweet."}}

#Now answer this:
#Product Information: {text}"""

In [12]:
#pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=10000)
#result = pipe(f"Prompt: {prompt} \n Response:") # result = pipe(f"Prompt: {prompt} \n Response:")
#generated_text = result[0]['generated_text']

In [22]:
#rm -r ./OrijinalLlama3.1-8B-Tuned

In [14]:
trainer.save_model("Peft-Deneme")
tokenizer.save_pretrained("Peft-Tokenizer")

In [1]:
#from peft import AutoPeftModelForCausalLM

#trained_model = AutoPeftModelForCausalLM.from_pretrained(
    #"Peft-Deneme",
    #low_cpu_mem_usage=True,
#).to('cuda')

In [20]:
text = "soy beans for your healty and delicious meals"
prompt = f"""
You are extracting product title and description from given text and rewriting the description and enhancing it when necessary.
Always give response in the user's input language.
Always answer in the given json format. Do not use any other keywords. Do not make up anything.
Explanations should contain at least three sentences each.

Json Format:
{{
"title": "<title of the product>",
"description": "<description of the product>"
}}

Examples:

Product Information: Rosehip Marmalade, keep it cold
Answer: {{"title": "Rosehip Marmalade", "description": "You should store this delicisious roseship marmelade in cold conditions. You can use it in your breakfasts and meals."}}

Product Information: Blackberry jam spoils in the heat
Answer: {{"title": "Blackberry Jam", "description": "Please store it in cold conditions. Recommended to be consumed at breakfast. Very sweet."}}

Now answer this:
Product Information: {text}"""

In [21]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=10000)
result = pipe(f"Prompt: {prompt} \n Response:") # result = pipe(f"Prompt: {prompt} \n Response:")
generated_text = result[0]['generated_text']